### Churn Project - Part C

In [1]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

import findspark
findspark.init("spark-3.5.1-bin-hadoop3")
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import Row
from pyspark.sql import functions


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,566 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,639 kB]
Get:13 http://security.ubuntu.com/ubuntu jam

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/MyDrive/BDA spark witty"
!ls "/content/drive/MyDrive/BDA spark witty"

appl_stock.csv		       Churn_ratings	 people.json	sales_info.csv
churn_location_and_rating.csv  NORTHWND_ord.csv  rating_df.csv	walmart_stock.csv


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType
from pyspark.sql.functions import udf, col, cast, when, lit, count, isnull, trim, countDistinct, avg, format_number, regexp_replace

In [6]:
df = spark.read.csv(path + '/churn_location_and_rating.csv',header = True, inferSchema = True)

In [7]:
df.describe().show()

+-------+--------------------------+
|summary|customerID;Location;Rating|
+-------+--------------------------+
|  count|                     24858|
|   mean|                      NULL|
| stddev|                      NULL|
|    min|      "id10000;christch...|
|    max|      id24334;&#321;ód&...|
+-------+--------------------------+



In [8]:
def split_row(row):

  row_string = row[len('Row(customerID;Location;Rating='):-1]

  values = [value.strip('"') for value in row_string.split(';')]
  return values

new_df = [split_row(str(row)) for row in df.collect()]

In [9]:
import pandas as pd

pandas_df = pd.DataFrame(new_df)
result = pandas_df.to_dict()
rating_df = pd.DataFrame(result)
rating_df = rating_df.rename(columns={0: 'customerid',
                                      1: 'city',
                                      2: 'state',
                                      3: 'rating',
                                      4: 'country',
                                      5: 'extra_column'})
rating_df['country_upper'] = rating_df['country'].str.upper()
rating_df.head()

,customerid,city,state,rating,country,extra_column,country_upper
0,"'""id7044",nyc,new york,9.0,"usa""""""'",None,"USA""""""'"
1,"'""id7045",stockton,california,5.0,"usa""""""'",None,"USA""""""'"
2,"'""id7046",moscow,yukon territory,5.0,"russia""""""'",None,"RUSSIA""""""'"
3,"'""id7047",porto,v.n.gaia,9.0,"portugal""""""'",None,"PORTUGAL""""""'"
4,"'""id7048",farnborough,hants,8.0,"united kingdom""""""'",None,"UNITED KINGDOM""""""'"


In [10]:
rating_df = rating_df.drop(columns=['extra_column'])
rating_df = rating_df.drop(columns=['country'])

rating_df = rating_df.drop_duplicates(["customerid"])

rating_df['customerid'] = rating_df['customerid'].str.replace('"', '')
rating_df['country_upper'] = rating_df['country_upper'].str.replace('"', '')

rating_df['customerid'] = rating_df['customerid'].str.replace("'", '')
rating_df['country_upper'] = rating_df['country_upper'].str.replace("'", '')

rating_df.rename(columns={'country_upper': 'country'}, inplace=True)

rating_df.head()

,customerid,city,state,rating,country
0,id7044,nyc,new york,9.0,USA
1,id7045,stockton,california,5.0,USA
2,id7046,moscow,yukon territory,5.0,RUSSIA
3,id7047,porto,v.n.gaia,9.0,PORTUGAL
4,id7048,farnborough,hants,8.0,UNITED KINGDOM


In [11]:
spark_df = spark.createDataFrame(rating_df)

def clean_column(df, column_name):
    df = df.withColumn(column_name,
                       when((df[column_name].isNull()) |
                            (df[column_name] == "") |
                            (df[column_name] == " ") |
                            (df[column_name].rlike(r'\b(?:n/a|[*]|[0-9])\b')), "unknown")
                       .otherwise(df[column_name]))

    df = df.withColumn(column_name, regexp_replace(column_name, "city", ""))
    df = df.withColumn(column_name, regexp_replace(column_name, "-", " "))

    return df


spark_df = clean_column(spark_df, "city")
spark_df = clean_column(spark_df, "state")

rating_df = spark_df.toPandas()
rating_df.head(1000)

,customerid,city,state,rating,country
0,id7044,nyc,new york,9.0,USA
1,id7045,stockton,california,5.0,USA
2,id7046,moscow,yukon territory,5.0,RUSSIA
3,id7047,porto,v.n.gaia,9.0,PORTUGAL
4,id7048,farnborough,hants,8.0,UNITED KINGDOM
...,...,...,...,...,...
995,id8043,teramo,abruzzo,5.0,ITALY
996,id8044,campbelltown,new south wales,6.0,AUSTRALIA
997,id8045,vilafranca,barcelona,10.0,SPAIN
998,id8046,tequesta,florida,8.0,USA


In [12]:
!pip install pycountry
import pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 24.8 MB/s eta 0:00:00


In [13]:
rating_df_spark = spark.createDataFrame(rating_df)

def findCountryFuzzy(country):
    try:
        result = pycountry.countries.search_fuzzy(country)
        print(result)
        return result[0].alpha_3
    except:
        return ("Unknown")

findCountryUDF = udf(findCountryFuzzy, StringType())
rating_df_spark = rating_df_spark.withColumn("country", findCountryUDF(col("country")))

rating_df_spark.show()

+----------+--------------+----------------+------+-------+
|customerid|          city|           state|rating|country|
+----------+--------------+----------------+------+-------+
|    id7044|           nyc|        new york|   9.0|    USA|
|    id7045|      stockton|      california|   5.0|    USA|
|    id7046|        moscow| yukon territory|   5.0|    RUS|
|    id7047|         porto|        v.n.gaia|   9.0|    PRT|
|    id7048|   farnborough|           hants|   8.0|    GBR|
|    id7049|  santa monica|      california|  10.0|    USA|
|    id7050|    washington|              dc|   3.0|    USA|
|    id7051|       timmins|         ontario|   2.0|    CAN|
|    id7052|    germantown|       tennessee|  10.0|    USA|
|    id7053|      albacete|       wisconsin|  10.0|    ESP|
|    id7054|     melbourne|        victoria|   3.0|    AUS|
|    id7055|    fort bragg|      california|   6.0|    USA|
|    id7056|     barcelona|       barcelona|   5.0|    ESP|
|    id7057|    mediapolis|            i

In [14]:
rating_df_spark = rating_df_spark.withColumn('customerid', regexp_replace(col("customerid"), "id", ""))
rating_df_spark.show()

+----------+--------------+----------------+------+-------+
|customerid|          city|           state|rating|country|
+----------+--------------+----------------+------+-------+
|      7044|           nyc|        new york|   9.0|    USA|
|      7045|      stockton|      california|   5.0|    USA|
|      7046|        moscow| yukon territory|   5.0|    RUS|
|      7047|         porto|        v.n.gaia|   9.0|    PRT|
|      7048|   farnborough|           hants|   8.0|    GBR|
|      7049|  santa monica|      california|  10.0|    USA|
|      7050|    washington|              dc|   3.0|    USA|
|      7051|       timmins|         ontario|   2.0|    CAN|
|      7052|    germantown|       tennessee|  10.0|    USA|
|      7053|      albacete|       wisconsin|  10.0|    ESP|
|      7054|     melbourne|        victoria|   3.0|    AUS|
|      7055|    fort bragg|      california|   6.0|    USA|
|      7056|     barcelona|       barcelona|   5.0|    ESP|
|      7057|    mediapolis|            i

In [15]:
rating_schema = StructType([
    StructField("customerid", StringType(), nullable=False),
    StructField("rating", StringType(), nullable=False),
    StructField("city", StringType(), nullable=True),
    StructField("state", StringType(), nullable=True),
    StructField("country",StringType(),nullable=True)
])


rating_df_spark = rating_df_spark.withColumn("rating", col("rating").cast(FloatType()))
rating_df_spark = rating_df_spark.withColumn("customerid", col("customerid").cast(IntegerType()))

rating_df_spark = rating_df_spark.dropna(subset=['customerid', 'rating'])
rating_df_spark = rating_df_spark.select(rating_schema.fieldNames())

rating_df_spark.printSchema()

root
 |-- customerid: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)



In [16]:
rating_df_spark = rating_df_spark.withColumn("city", trim(col("city")))
rating_df_spark = rating_df_spark.withColumn("state", trim(col("state")))

In [18]:
output_path = "/content/drive/MyDrive/BDA spark witty/rating_df.csv"
rating_df_spark.coalesce(1).write.mode("overwrite").csv(output_path, header=True)
rating_df_spark.show()

+----------+------+--------------+---------------+-------+
|customerid|rating|          city|          state|country|
+----------+------+--------------+---------------+-------+
|      7044|   9.0|           nyc|       new york|    USA|
|      7045|   5.0|      stockton|     california|    USA|
|      7046|   5.0|        moscow|yukon territory|    RUS|
|      7047|   9.0|         porto|       v.n.gaia|    PRT|
|      7048|   8.0|   farnborough|          hants|    GBR|
|      7049|  10.0|  santa monica|     california|    USA|
|      7050|   3.0|    washington|             dc|    USA|
|      7051|   2.0|       timmins|        ontario|    CAN|
|      7052|  10.0|    germantown|      tennessee|    USA|
|      7053|  10.0|      albacete|      wisconsin|    ESP|
|      7054|   3.0|     melbourne|       victoria|    AUS|
|      7055|   6.0|    fort bragg|     california|    USA|
|      7056|   5.0|     barcelona|      barcelona|    ESP|
|      7057|   5.0|    mediapolis|           iowa|    US